# DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [1]:
from src.StreamPort.device.DeviceEngine import DeviceEngine
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

In [2]:
#specify path to get analyses from
base_dir = r'C:\Users\PC0118\Desktop\ExtractedSignals'

Creates an empty DeviceEngine object and prints it

In [ ]:

dev = DeviceEngine(source = base_dir)
dev.print()

DeviceEngine object without an explicitly provided source performs all capabilities on files within the current working directory.

In [ ]:
dev1 = DeviceEngine()
dev1.print()
print(dev1._source)
del dev1

# ProjectHeaders Class

Add project headers. They can be passed as ProjectHeaders objects or dict

In [ ]:
dev.add_headers(headers = {'name': 'Pressure Curve Analysis', 'author': 'Sandeep H.'})
dev.print()

# DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the details of an Analysis for each individual device.

In [ ]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()

 
DeviceEngine's find_analyses() method returns a DeviceAnalysis Object or a list of DeviceAnalysis objects, besides printing the dataframes for each unique Method, paired with the metadata(Date, Runtime) for each curve.

This method makes use of the source variable to accept a path to a directory containing analyses as an argument and find analyses from the target path.

The path can refer to a directory containing data for specific groups of experiments "210812_Gem 2021-08-12 09-49-10" or one such experiment containing its own set of method-related analysis data "210812_Gem--005.D", "210812_Gem--007.D", ..



Read analysis objects from engine.

In [ ]:
analyses = dev.find_analyses()

Each DeviceEngine object has an attribute _method_ids that records all methods encountered in the analysis of the current Device.

In [ ]:
print(dev._method_ids)

And an attribute _history to hold data on all experiments related to this device.

In [ ]:
dev.print()

Add analyses objects that were found using find_analyses() to current device records.

Add analyses in the form of individual DeviceAnalysis objects or a list of such objects.

In [10]:
dev.add_analyses(analyses)

In [ ]:
dev.print()

In [ ]:
for ana in dev._analyses:
    print("\n")
    print("Analysis Object : \n")
    print(f"Analysis : {ana.print()}")
    print("Data of Analysis : \n")
    print(ana.data)
    print("\n")

# Plot Analyses

DeviceEngine's *plot_analyses()* and *plot_results()* calls each analysis object's respective *plot()* function after dynamically grouping related analyses. 
Grouping is done on the basis of unique method id's paired with unique experiment dates.
User can set the 'group_by'(str) argument to control how the data is grouped. Defaults to 'method', otherwise 'date'

Plot analyses by calling inbuilt plot function and passing each object's index as argument

Plot analyses by word or subword present in analysis date

In [ ]:
dev.plot_analyses('Pac', group_by='date')

Plot all available analyses by omitting 'analyses' argument
Group by defaults to 'method'

In [ ]:
dev.plot_analyses('Gem', group_by='method')

# ProcessingSettings - Feature Extraction

Create a new ProcessingSettings object 

In [15]:
from src.StreamPort.device.DeviceProcSettings import ExtractPressureFeatures

'weighted' argument of ExtractPressureFeatures object can be used to control whether the pressure curves should first be transformed by calculating percentage change between adjacent datapoints.
Defaults to False, in which case feature extraction is performed on the raw pressure curves.

In [16]:
settings = ExtractPressureFeatures(weighted=True)

Add processing settings

In [ ]:
dev.add_settings(settings)
dev.print()

Now we run the settings to extract pressure features after adding analyses.

In [18]:
pressure_features = settings.run(dev)

In [ ]:
print(pressure_features)

Add the extracted features to the results (dict) attribute

In [20]:
dev.add_results(pressure_features)

Retrieve the stored results associated with the current object.

# ProcessingSettings - Seasonal Decomposition

Create a new ProcessingSettings object to extract seasonal components from analyses.

In [21]:
from src.StreamPort.device.DeviceProcSettings import DecomposeCurves

*'period' argument of DecomposeCurves is used to control the window size over which the features are calculated. Defaults to 30 here.

In [22]:
curve_decompose = DecomposeCurves(period=30)

In [ ]:
dev.add_settings(curve_decompose)
dev.print()

In [ ]:
seasonal_components = curve_decompose.run(dev)
print(seasonal_components)

In [25]:
dev.add_results(seasonal_components)

In [ ]:
dev.get_results(-1)

#Each .D folder is an analysis with timestamp

Latest entry in analyses contains most up to date results

# ProcessingSettings - Fourier Transformation

Create a new ProcessingSettings object to perform Fast Fourier Analysis on raw curve and seasonal component of analyses time decomposition.

In [27]:
from src.StreamPort.device.DeviceProcSettings import FourierTransform

In [28]:
fourier_transform = FourierTransform()

In [ ]:
dev.add_settings(fourier_transform)
dev.print()

In [ ]:
transformed_seasonal = fourier_transform.run(dev)
print(transformed_seasonal)

In [31]:
dev.add_results(transformed_seasonal)

In [ ]:
dev.get_results(-1)

scaled results are unavailable since data has not been scaled yet

Adding features before scaling:
scale_features() calls add_extracted_features() before grouping and scaling data.

add_extracted_features() introduces new features that were extracted from the behaviour of the seasonal and noise components of the raw curves in the frequency domain. These frequencies were binned and averaged in different time-windows and added as features.

Additional features added were Idle time of the batch, error in defined vs. measured runtime.

In [ ]:
dev.print()

# ProcessingSettings - Feature Scaling  


Scale extracted and engineered features to improve the quality of the information we get from them. These prove more useful when visually analysing data

In [34]:
from src.StreamPort.device.DeviceProcSettings import Scaler

User selects the type of scaler to be used from preloaded options : 'minmax', 'std'(Standard), 'robust', 'maxabs', 'norm'(Normalizer).
Scaler defualts to Normalizer in the absence of an argument.

'replace' argument allows user to replace existing features with scaled features or to create a new entry instead. Defaults to False.

In [35]:
feature_scaler = Scaler(parameters='std')

In [ ]:
dev.add_settings(feature_scaler)
dev.print()

In [ ]:
scaled_features = feature_scaler.run(dev)

In [38]:
dev.add_results(scaled_features)

In [ ]:
dev.print()

# Plot Results

Plot the computed results of feature extraction for chosen results based on user input to select *base* to extract base features, *decompose* for seasonal decomposition, fourier *transform* 

User may also plot the raw pressure curves by omitting the 'features' argument, indicating that the *results* of feature extraction are not to be plotted, just the curves.

In [ ]:
#this_method = dev._method_ids[6]
this_method = 'Pac' 
print(this_method)

'group_by' allows user to group data either by 'date' or 'method':
1. 'date' prepares data with weight on experiment date. So matching methods on different dates will not be grouped.
2. 'method' prepares data purely on method and groups all available data for the given method.

In [ ]:
dev.plot_results(this_method)

Select features to plot. Setting 'scaled' argument allows to toggle plots of scaled features or unscaled. Defaults to True.


In [ ]:
dev.plot_results(results = this_method, features ='base', scaled=True, transpose=True, group_by='method', interactive=False)

In [ ]:
dev.plot_results(results = this_method, features ='base', transpose=False, interactive=False)

use 'interactive' argument to toggle between static and interactive plots

setting type to 'box' enables a box plot of the data. Available options are 'box' and 'scatter' by default

In [ ]:
dev.plot_results(results = this_method, features ='transform')

# MachineLearning - Isolation Forest for preliminary classification  

ADD CLASS LABELS TO ANALYSIS OBJECTS AFTER FEATURE ANALYSIS. FIRST ANALYSIS '001-blank' is assigned a separate class of ML operations due to it being a systematic fault.

classify() dynamically assigns class labels through MLEngine's make_iso_forest() to all analyses encountered and classified

First, create a MachineLearningEngine object to enable ML ops on prepared data.

In [45]:
from src.StreamPort.ml.MachineLearningEngine import MachineLearningEngine
from src.StreamPort.ml.MachineLearningAnalysis import MachineLearningAnalysis
from src.StreamPort.ml.MachineLearningProcessingSettings import MakeModelIsoForest
from src.StreamPort.ml.MachineLearningProcessingSettings import MakeModelPCASKL

In [46]:
ml_engine = MachineLearningEngine()

random_state(int) argument can be specified to reproduce results. Defaults to None, sets a random seed.

In [47]:
iso_forest = MakeModelIsoForest(dev, random_state=22)

In [ ]:
ml_engine.add_settings(iso_forest)
ml_engine.print()

In [ ]:
method_objects = iso_forest.run(ml_engine)

# MachineLearning - PCA

make_iso_forest() of MLEngine class automatically creates sub-objects of MLEngine class for each encountered group of analyses per unique method after performing iso_forest and plotting results. Can be modified to save results later

In [50]:
pca = MakeModelPCASKL(n_components = 2, center_data= True)

In [ ]:
import webbrowser
for obj in method_objects:
    obj.add_settings(pca)
    obj.print()
    pca_scores = pca.run(obj)
    obj.add_results(pca_scores)
    obj.plot_pca()
    
    webbrowser.open('pca_scores_plot.html')
    webbrowser.open('pca_loadings_plot.html')

# Reproduce everything here on Orange and then try 26k data

# XML file manipulation for real-time classification and maintenance

In [52]:
import xml.etree.ElementTree as ET

Future implementation will allow to scan for actuals in a directory

In [53]:
xml_file = r'C:\Users\PC0118\Desktop\Chemstation Actuals\actuals 9.8.2024 10_18_9-943.xml'

In [54]:
tree = ET.parse(xml_file)
root = tree.getroot()

Traverse from root to end nodes and find relevant status information to build a dataframe out of.

In [ ]:
#second child of root contains actuals, first child holds schematics for data
print(root[0].tag, root[0].attrib)
diffgrams = root[1]
print(len(diffgrams))
print(diffgrams.tag, diffgrams.attrib, diffgrams.text)


import pandas as pd
#dataframe to hold data in xml file initiated with list of entries and sample names identified by timestamp 
diffgram_df = pd.DataFrame()
samples = []

feature = []

num_observations = len(diffgrams[0])
print('Observations', num_observations)

for element in diffgrams[0]:
        print(element.tag, element.attrib, element[0].text)

        if element[0].text in samples:
                feature = pd.DataFrame(feature, index=[f'Analysis - {sample}' for sample in samples])
                diffgram_df = pd.concat([diffgram_df, feature], axis = 1)
                feature = []
                samples = []
        feature.append({element.tag : element.get('{urn:schemas-microsoft-com:xml-diffgram-v1}id')})
        samples.append(element[0].text)
if feature != [] or samples != []:
        feature = pd.DataFrame(feature, index=[f'Analysis - {sample}' for sample in samples])
        diffgram_df = pd.concat([diffgram_df, feature], axis = 1)


In [ ]:
print(diffgram_df)

# Dashboard

In [57]:
#packages to create a dashboard
import dash
from dash import dcc
from dash import html 
from dash.dependencies import Input, Output

Set up divisions with the option to select the information to be displayed

# something off here

In [58]:
app = dash.Dash(__name__)
app.layout =html.Div([

                        html.Div([  
                        html.H1('Title', style={'text-align' : 'center'}),


                        dcc.RadioItems(
                                        id='radio-items',
                                        options=[
                                                    {'label' : 'Curves', 'value' : ''},
                                                    {'label' : 'Features', 'value' : 'base'},
                                                    {'label' : 'Decomp', 'value' : 'decompose'},
                                                    {'label' : 'Transform', 'value' : 'transform'}
                                                ],
                                        value=''   #default
                                       ),
                                       html.Div(id='output-container',
                                                style={
                                                    'backgroundColor': '#f9f9f9',
                                                    'border': '1px solid #ccc',
                                                    'padding': '20px',
                                                    'borderRadius': '5px',
                                                    'boxShadow': '2px 2px 12px rgba(0, 0, 0, 0.1)'
                                                    }
                                                )
                                ]),

                        html.Div([
                        dcc.DatePickerRange(
                            id='date-picker-range',
                            start_date='2023-01-01',
                            end_date='2023-12-31',
                            display_format='YYYY-MM-DD'
                        )
                        ], style={'border': '1px solid black', 'padding': '10px', 'margin': '10px'})

                    ]) 

            


In [ ]:
#import webbrowser
@app.callback(    
    Output('output-container', 'children'),
    Input('radio-items', 'value')
)
def update_graph(value):
    dev.plot_results('Pac', features=value)
    #webbrowser.open('plot.html')
    return     

if __name__ == '__main__':
    app.run_server(debug=True)